In [1]:
import tensorflow as tf
import pickle

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist as fmnist

In [2]:
import cv2
import numpy as np
from numpy import dstack 
import pandas as pd
from pandas import read_csv
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import math

---
---

#FedAvg using MLP

### For N clients FedAvG

In [3]:
def avgWeights(scaledWeights):
    avg = list()
    for weight_list_tuple in zip(*scaledWeights):
        layer_mean = tf.math.reduce_sum(weight_list_tuple, axis=0)
        avg.append(layer_mean)
    return avg

def FedAvg(models):  
    scaledWeights = []
    for m in models:
        scaledWeights.append(getScaledWeight(m, FLAccuracyDF.loc[m]['Weightage']))
    avgWeight = avgWeights(scaledWeights)
    return avgWeight

def define_mod2(input_shape):
  model = Sequential()
  model.add(Dense(512, input_shape=input_shape, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  # Configure the model and start training
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [5]:
import numpy as np
feature_vector_length = 784
# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Load the data
def load_dataset2(n_clients=3,permute=False):

  client_datasets = {} # defining local datasets for each client

  (X_train, Y_train), (X_test, Y_test) = fmnist.load_data()

 
  X_train = X_train.reshape(X_train.shape[0], feature_vector_length)
  X_test = X_test.reshape(X_test.shape[0], feature_vector_length)

  if permute==True:
    permutation_indexes = np.random.permutation(len(X_train))
    X_train  = X_train[permutation_indexes]
    Y_train = Y_train[permutation_indexes]

  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train /= 255
  X_test /= 255

  # Convert target classes to categorical ones
  Y_train = to_categorical(Y_train)
  Y_test = to_categorical(Y_test)

  for i in range(n_clients):
    client_datasets[i] = [X_train[i*(len(X_train)//n_clients):i*(len(X_train)//n_clients)+(len(X_train)//n_clients)],Y_train[i*(len(Y_train)//n_clients):i*(len(Y_train)//n_clients)+(len(Y_train)//n_clients)]]


  return client_datasets  


Feature shape: (784,)


In [6]:
def load_dataset3(n_clients=3,permute=False):

  client_datasets = {} # defining local datasets for each client

  (X_train, Y_train), (X_test, Y_test) = fmnist.load_data()

 
  X_train = X_train.reshape(X_train.shape[0], feature_vector_length)
  X_test = X_test.reshape(X_test.shape[0], feature_vector_length)

  if permute==True:
    permutation_indexes = np.random.permutation(len(X_train)//n_clients)
    X_train  = X_train[permutation_indexes]
    Y_train = Y_train[permutation_indexes]

  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train /= 255
  X_test /= 255

  # Convert target classes to categorical ones
  Y_train = to_categorical(Y_train)
  Y_test = to_categorical(Y_test)

  return X_train,X_test,Y_train,Y_test


In [7]:
def testing():

  (_, _), (X_test, Y_test) = fmnist.load_data() 
  X_test = X_test.reshape(X_test.shape[0], feature_vector_length)
  X_test = X_test.astype('float32')
  X_test /= 255
  # Convert target classes to categorical ones
  Y_test = to_categorical(Y_test)
  return X_test,Y_test  


In [8]:

  
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

def f_to_i(x,scale = 1<<32):
  # print(f" x : {x}")
	 # embed float value onto the integer ring
  if x < 0:
    if pow(2,64) - abs(x)*(scale) > (pow(2,64) - 1):
      # print(pow(2,64) - abs(x)*(scale))
      return np.uint64(0)
    x = pow(2,64) - np.uint64(abs(x)*(scale))
   
  else:
    x = np.uint64(scale*x)
  
  return np.uint64(x)

				


def i_to_f(x,scale = 1<<32):
  l=64
  t = x > ( pow(2,(l-1)) -1 )
  if t:
    x = pow(2,l) - x
    y = np.uint64(x)
    y = np.float32(y*(-1))/scale
    # y = np.array(y,dtype=np.float64)/scale
  else:
    y = np.float32(np.uint64(x))/scale
    # y = np.array(y,dtype=np.float64)/scale

  return y


f_to_i_v = np.vectorize(f_to_i)
i_to_f_v = np.vectorize(i_to_f)

In [10]:

def fed_avg_pipe(N_clients=10,num_of_iterations=3,num_servers=3):
   
  client_datasets = load_dataset2(N_clients,permute=True)
  LD = len(client_datasets[0][0])//num_of_iterations

  X_test,Y_test = testing()
  avg_wts=0
  for iteration in range(num_of_iterations):

    list_of_weights = []
    for j in range(N_clients):
      if iteration==0:
        model_c = define_mod2(input_shape)
      else:
        model_c = define_mod2(input_shape)
        model_c.set_weights(avg_wts)


      X_train,Y_train = client_datasets[j][0][(iteration*LD) :(iteration*LD)+LD],client_datasets[j][1][(iteration*LD) :(iteration*LD)+LD]
      
      model_c.fit(X_train, Y_train, epochs=4, batch_size=100, verbose=1, validation_split=0.1)

      client_models[j]= model_c
      W_c= model_c.get_weights()
      list_of_weights.append(W_c)
      
    ################## Simulating Secret Sharing Framework Setting ####################################


    final_weights=[None]*len(list_of_weights[0])
    all_servers=[{}]*num_servers
    for m in range(N_clients):
      layer_dict,layer_shape,shares_dict={},{},{}
      data=list_of_weights[m]
      no_of_layers=len(data)
      for i in range(len(data)):
          layer_dict[i]=data[i]
          layer_shape[i]=data[i].shape
          
      for i in range(no_of_layers):
          shares_dict[i]=np.random.randint(1000,size=(num_servers,)+layer_shape[i],dtype=np.uint64)
          x = f_to_i_v(layer_dict[i])
          for k in range(0,num_servers-1):
              shares_dict[i][k]=np.random.randint(1000,size=layer_shape[i],dtype=np.uint64)
              x = x - shares_dict[i][k]
          shares_dict[i][num_servers-1] = x
                              
      for i in range(num_servers):
        if m==0:
          for j in range(len(shares_dict)):
            shape=shares_dict[j][0].shape
            all_servers[i][j]=np.random.random_sample((N_clients,)+shape)
            all_servers[i][j][m]=shares_dict[j][0] 
        else:
          for j in range(len(shares_dict)):
            all_servers[i][j][m]=shares_dict[j][i]


    for i in range(num_servers):
      for j in range(len(all_servers[0])):
        all_servers[i][j] = f_to_i_v(i_to_f_v(all_servers[i][j])/np.float32(N_clients))
       
    for i in range(num_servers):
      for j in range(len(all_servers[0])):
        for k in range(N_clients):
          if i==0 and k==0:
            final_weights[j]=all_servers[i][j][k]
          else:
            final_weights[j]=final_weights[j]+all_servers[i][j][k]

    for j in range(len(final_weights)):
      final_weights[j]=i_to_f_v(final_weights[j])
    avg_wts=final_weights
    avg_model=define_mod2(input_shape)
    avg_model.set_weights(avg_wts)
################## Simulating Secret Sharing Framework Setting ####################################

  test_results = avg_model.evaluate(X_test, Y_test, verbose=1)
  return test_results[1]

In [ ]:
test_accuracies = []
for y in range(4,5,1):
  for q in range(2,5,1):
    res = fed_avg_pipe(N_clients=q,num_of_iterations=y,num_servers=3)
    test_accuracies.append(res)
    #print(f" {q} :  {res}")
  print(f"\n\n\n With {y} iterations the test_acc is {test_accuracies}")


In [12]:
print(test_accuracies)

[0.8240000009536743, 0.6949999928474426, 0.4869999885559082]
